# Task 12 - KMeans
## Pattern Recognition and Machine Learning

Copy and import needed files/methods from previous assignment to this directory. 
Adding path to the previous assignment is not sufficient. Upload system
requires your code to be self contained.

In [ ]:
# uncomment following for interactive matplotlib
# %matplotlib notebook
%load_ext autoreload
%autoreload 2

from kmeans import *
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import imageio

## init

In [ ]:
def unwrap(data):
    """
    Simple "hack" for preparing data from *.mat files
    """
    try:
        while (len(data) == 1) and (len(data.shape) > 0):
            data = data[0]
        for key in list(data.dtype.names):
            data[key] = unwrap(data[key])
    except:
        pass
    return data

def ndarray2dict(data, indexes=None):
    outputs = {}
    for key in list(data.dtype.names):
        value = unwrap(data[key])
        try:
            if len(value.shape) > 0:
                value = np.atleast_2d(value)
        except:
            pass
        outputs[key] = value
    if indexes is not None:
        for key in indexes:
            outputs[key] -= 1
    return outputs

#### Load data

In [ ]:
data = scipy.io.loadmat("image_data.mat") # Images of letters
images = np.atleast_2d(unwrap(data["images"]))
data = scipy.io.loadmat("mnist_image_data.mat") # Images of numbers from the MNIST dataset
mnist_images = np.atleast_2d(unwrap(data["mnist_images"]))
im = imageio.imread('geeks.png')

#### Init random generator

In [ ]:
np.random.seed(0)
random.seed(0) # separate module seed

#### K-means (HLT letters)

In [ ]:
print('K-means, K=3...')
x = compute_measurements(images.astype(np.float64))
k = 3

c, means, sq_dist = k_means(x, k, 100, show=False)
show_clusters(x, c, means);
show_clustered_images(images, c)
show_mean_images(images, c);

## K-means (numbers)

In [ ]:
print('K-means, K=10...')
x = compute_measurements(mnist_images)
k = 10
c, means, _ = k_means(x, k, np.inf)
show_clusters(x, c, means)
show_clustered_images(mnist_images, c)
show_mean_images(mnist_images, c)

## Avoid local minima

In [ ]:
print('K-means with multiple trials...')
x = compute_measurements(images)
k = 3
c_m, means_m, x_dists_m = k_means_multiple_trials(x, k, 10, np.inf, False)
show_clusters(x, c_m, means_m)
show_clustered_images(images, c_m)

## Random initialization (try this block multiple times)


In [ ]:
print('K-means, K=4...')
x = gen_kmeanspp_data()
k = 4
means = x[:, random.sample(range(x.shape[1]), k)]
show_clusters(x, np.ones([1, x.shape[1]]), means)
plt.title('K-means initialization (random)')
plt.savefig('random_initialization.png')
c, means, _ = k_means(x, k, np.inf, False, means)
show_clusters(x, c, means)
plt.title('K-means results (random initialization)')

## K-means++ initialization  (try this block multiple times)

In [ ]:
print('K-means++, K=4...')
x = gen_kmeanspp_data()
k = 4
means = k_meanspp(x, k)
show_clusters(x, np.ones([1, x.shape[1]]), means)
plt.title('K-means initialization (K-Means++)')    
plt.savefig('kmeanspp_initialization.png')
c, means, _ = k_means(x, k, np.inf, False, means)
show_clusters(x, c, means)
plt.title('K-means results (K-means++ initialization)')

## K-means Application - Color Quantization

In [ ]:
im_q = quantize_colors(im, 8)
plt.imshow(im_q)
plt.savefig('geeks_quantized.png')